In [110]:
%matplotlib inline
from nltk.stem.wordnet import WordNetLemmatizer
import string
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import re
import os
import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from nltk.corpus import gutenberg, stopwords
from collections import Counter
os.getcwd()
os.chdir('/Users/kylehoward/desktop/thinkful/thinkful-2') 
os.getcwd()

'/Users/kylehoward/Desktop/Thinkful/Thinkful-2'

In [111]:
'''
This project is aimed at performing natural language processing on a corpus of 100 scientific articles. All articles 
come from www.sciencenews.org. We are using word2vec in attempt to identify similar words, tf-idf to attempt to find 
similar articles, and also attempting group similar articles with latent dirichlet allocation. 

- w2v seems successful in identifying similar and different words

- tf-idf seems to overfit 

- LDA is very successful with certain groups and poor with others

This is a work in progress
'''

'\nThis project is aimed at performing natural language processing on a corpus of 100 scientific articles. All articles \ncome from www.sciencenews.org. We are using word2vec in attempt to identify similar words, tf-idf to attempt to find \nsimilar articles, and also attempting group similar articles with latent dirichlet allocation. \n\n- w2v seems successful in identifying similar and different words\n\n- tf-idf seems to overfit \n\n- LDA is very successful with certain groups and poor with others\n\nThis is a work in progress\n'

In [112]:
# All articles are from https://www.sciencenews.org
# Load Test
f = open("Article_2.txt", "r")
f = f.read()
#print(f)

In [113]:
# Getting 100 unique article names
text_list = []
for x in range(101):
    y = "Article_" + str(x) + ".txt"
    text_list.append(y)
# Adjust list because there is no article '0'
text_list = text_list[1:]
print(text_list)

['Article_1.txt', 'Article_2.txt', 'Article_3.txt', 'Article_4.txt', 'Article_5.txt', 'Article_6.txt', 'Article_7.txt', 'Article_8.txt', 'Article_9.txt', 'Article_10.txt', 'Article_11.txt', 'Article_12.txt', 'Article_13.txt', 'Article_14.txt', 'Article_15.txt', 'Article_16.txt', 'Article_17.txt', 'Article_18.txt', 'Article_19.txt', 'Article_20.txt', 'Article_21.txt', 'Article_22.txt', 'Article_23.txt', 'Article_24.txt', 'Article_25.txt', 'Article_26.txt', 'Article_27.txt', 'Article_28.txt', 'Article_29.txt', 'Article_30.txt', 'Article_31.txt', 'Article_32.txt', 'Article_33.txt', 'Article_34.txt', 'Article_35.txt', 'Article_36.txt', 'Article_37.txt', 'Article_38.txt', 'Article_39.txt', 'Article_40.txt', 'Article_41.txt', 'Article_42.txt', 'Article_43.txt', 'Article_44.txt', 'Article_45.txt', 'Article_46.txt', 'Article_47.txt', 'Article_48.txt', 'Article_49.txt', 'Article_50.txt', 'Article_51.txt', 'Article_52.txt', 'Article_53.txt', 'Article_54.txt', 'Article_55.txt', 'Article_56.txt', 

In [114]:
# Reading in all articles
article_list = []
for x in text_list:
    f = open(x, "r")
    f = f.read()
    article_list.append(f)
#print(article_list[:10])

In [115]:
# Cleaning data. Removing all characters except for letters. Removing whitespace 
w2v = ""
for art in article_list:
    work = art
    w2v = w2v + work


w2v = re.sub("[^a-zA-Z]"," ", w2v)
w2v = ' '.join(w2v.split())
len(w2v)

356922

In [116]:
# Tokenizing 
nlp = spacy.load('en')
w2v_tok = nlp(w2v)
#print(w2v_tok)

In [117]:
# Converting tokens to sentences and lemmas. Assuring punctuation is scrubbed
sentences = []
for sentence in w2v_tok.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences.append(sentence)


#print(sentences)
print('We have {} tokens.'.format(len(w2v)))

We have 356922 tokens.


In [118]:
#------------------------------------------CBOW word2vec-----------------------------------------------
# Attempting to group words together based on proximity. Using CBOW due to smaller corpus
cbow_model = word2vec.Word2Vec(
    sentences,     
    min_count=5,   # Minimum word count threshold.
    window=10,     # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)


In [119]:
# word2vec analysis 
vocab = cbow_model.wv.vocab.keys()

# Testing to see which words it will group with 'planet'
print('Similarity to planet')
print(cbow_model.wv.most_similar('planet'))

# Testing for similarity 
print('Cosine similatrity analysis:')
print('study / research')
print(cbow_model.wv.similarity('study', 'research'))
print('atom / molecule')
print(cbow_model.wv.similarity('atom', 'molecule'))

# Testing to see if it will leave out the right word (looking for 'dinosaur')
print('Which doesnt fit?')
print('bat / bird / dinosaur / bee')
print(cbow_model.doesnt_match("bat bird dinosaur bee".split()))

Similarity to planet
[('space', 0.9998832941055298), ('physicist', 0.9998636245727539), ('spacecraft', 0.9998591542243958), ('long', 0.9998512864112854), ('signal', 0.9998338222503662), ('life', 0.9998282790184021), ('structure', 0.9998061656951904), ('real', 0.9998008012771606), ('galaxy', 0.9997961521148682), ('send', 0.9997789859771729)]
Cosine similatrity analysis:
study / research
0.999840003593
atom / molecule
0.999710865559
Which doesnt fit?
bat / bird / dinosaur / bee
dinosaur


In [108]:
#-------------------------------------------tfidf-------------------------------------------------

X_train, X_test = train_test_split(article_list, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.25,          # drop words that occur in more than a quarter of the articles
                             min_df=2,             # only use words that appear at least twice
                             stop_words='english', # picking list of stop words
                             lowercase=True,       # convert everything to lower case 
                             use_idf=True,         # use inverse document frequencies in our weighting
                             norm=u'l2',           # Treat longer and shorter paragraphs equally
                             smooth_idf=True       # Prevent divide-by-zero errors
                            )


# Applying the vectorizer
article_list_tfidf = vectorizer.fit_transform(article_list)
print("Number of features: %d" % article_list_tfidf.get_shape()[1])

X_train_tfidf, X_test_tfidf= train_test_split(article_list_tfidf, test_size=0.4, random_state=0)


# Reshape the vectorizer output to be legible 
X_train_tfidf_csr = X_train_tfidf.tocsr()

# number of articles
n = X_train_tfidf_csr.shape[0]

# Making a list of dictionaries, one per article
tfidf_bypara = [{} for _ in range(0,n)]

# List of features
terms = vectorizer.get_feature_names()

# for each article, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]


print('Original article:', X_train[5])
print('Tf_idf vector:', tfidf_bypara[5])

Number of features: 3383
Original article: ﻿The deep waters of the East Pacific hold an unprepossessing treasure trove: potato-sized lumps of rock that contain valuable metals such as manganese, cobalt and copper. Turns out, such “manganese nodules” are home to another kind of goody: a species of sponge never before seen, researchers report online September 24 in Systematics and Biodiversity. These newly discovered nodule-dwellers may help scientists monitor the impact of future deep-sea mining.
Little is known about life in the abyssal depths of the ocean, 4,000 to 6,000 meters down. But the prospect of mining in those depths is looming: For example, the United Nation’s International Seabed Authority has granted 16 exploration contracts for mining manganese nodules. To track how mining will affect deep-sea ecosystems over time, scientists are eager to establish a baseline of existing biodiversity in regions such as the Clarion-Clipperton Zone (CCZ), an area in the eastern Pacific Ocea

In [109]:
# Applying SVD to reduce term document matrix into lower dimensional space (from 3417 to 260)
svd= TruncatedSVD(260)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data and projecting the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

# Checking variance explained
variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100) # Upon review seems like overfitting

# Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
# It seems like some similarities exist but needs refining
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

Percent variance captured by all components: 100.0
Component 0:
﻿Here are some of the things Kay Tye relishes: break dancing, rock-climbing, snowboarding, poker, raising her young daughter and son. These adrenaline-fueled activities all require basic skills. But true mastery — and the joy Tye finds in them — comes from improvisation. She boldly steps into a void, a realm where she has to riff, and trusts that a flash of insight will lead the way out.\nAs a 36-year-old neuroscientist studying how the brain creates experiences, Tye brings this mix of fearlessness and creativity to her lab, where it’s a key ingredient to her success. “Kay always finds this interesting twist,” says Leslie Vosshall, a molecular neurobiologist at Rockefeller University in New York City. Tye’s group at MIT investigates scientific questions in innovative ways, often with powerful results.\nThe goal, Tye says, is ambitious: to identify — in neuroscientific terms — the core of what makes us individuals. We all l

In [35]:
# Viewing the vectorizer output
print(X_train_tfidf_csr)

  (0, 0)	0.0404293192547
  (0, 1)	0.0204354194441
  (0, 12)	0.0221962627515
  (0, 13)	0.0204354194441
  (0, 14)	0.0690317775364
  (0, 16)	0.021098793607
  (0, 31)	0.0243904653549
  (0, 45)	0.0239064850676
  (0, 50)	0.032844770756
  (0, 57)	0.0404293192547
  (0, 60)	0.0287988463509
  (0, 61)	0.0221962627515
  (0, 87)	0.0358566867733
  (0, 113)	0.0358566867733
  (0, 136)	0.0260225085294
  (0, 215)	0.0260225085294
  (0, 222)	0.0404293192547
  (0, 235)	0.032844770756
  (0, 270)	0.0358566867733
  (0, 282)	0.0404293192547
  (0, 286)	0.0287988463509
  (0, 289)	0.032844770756
  (0, 294)	0.0342246435988
  (0, 303)	0.032844770756
  (0, 319)	0.094948411592
  :	:
  (59, 1845)	0.154864542621
  (59, 1943)	0.174613680133
  (59, 1999)	0.174613680133
  (59, 2001)	0.174613680133
  (59, 2040)	0.103251783944
  (59, 2147)	0.174613680133
  (59, 2233)	0.150899260754
  (59, 2236)	0.091125402703
  (59, 2361)	0.163491542667
  (59, 2362)	0.0958653570155
  (59, 2521)	0.154864542621
  (59, 2556)	0.148941626493
  (

In [85]:
#--------------------------------------------LDA for clustering------------------------------------------
# Attempting to group each article with Latent Dirichlet Allocation
# Cleaning data with a function
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    # Remove stopwords
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    # Remove punctuation
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    # Get lemmas for all words
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    # Remove numbers and outliers
    normalized_2 = re.sub("[^a-zA-Z]"," ", w2v)
    # Remove extra whitespace
    normalized_3 = ' '.join(normalized_2.split())
    return normalized
# Apply function
doc_clean = [clean(doc).split() for doc in article_list]    

In [86]:
dictionary = corpora.Dictionary(doc_clean)

In [87]:
# Creating the doc term matrix to use as our input
doc_term_matrix = [dictionary.doc2bow(article_list) for doc in article_list]

In [88]:
# Creating the word vectors
word_vectors = cbow_model.wv.syn0

In [89]:
# Building the model
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, 
               num_topics=10,        # With 100 articles I'm thinking 10% is a good test run
               id2word = dictionary, 
               alpha=1,              # number of topics
               passes=250)           # Number of passes over the corpus

/Users/kylehoward/Desktop/Thinkful/anaconda/lib/python3.6/site-packages/gensim/models/ldamodel.py:527: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)
/Users/kylehoward/Desktop/Thinkful/anaconda/lib/python3.6/site-packages/gensim/models/ldamodel.py:527: RuntimeWarning: invalid value encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [90]:
# Finding percent certainty for every category within each article
org = []
def get_org():
    for d in article_list:
        bow = dictionary.doc2bow(d.split())
        t = ldamodel.get_document_topics(bow)
        org.append(t)
        print(org)
    return org


In [91]:
# Creating a variable to iterate over
lst = get_org()   

[[(0, 0.1090186927085825), (1, 0.099510658029171292), (2, 0.10443083051301108), (3, 0.096818545489455302), (4, 0.10403733132551497), (5, 0.096666639869577853), (6, 0.097496353956395818), (7, 0.097416784973873599), (8, 0.089633012629827757), (9, 0.10497115050458986)]]
[[(0, 0.1090186927085825), (1, 0.099510658029171292), (2, 0.10443083051301108), (3, 0.096818545489455302), (4, 0.10403733132551497), (5, 0.096666639869577853), (6, 0.097496353956395818), (7, 0.097416784973873599), (8, 0.089633012629827757), (9, 0.10497115050458986)], [(0, 0.10009526477830923), (1, 0.11046698331687663), (2, 0.10334391188932507), (3, 0.10965892506057254), (4, 0.098800840328416795), (5, 0.085062693158313579), (6, 0.1046500872644716), (7, 0.087054635645778963), (8, 0.096706653012677821), (9, 0.10416000554525781)]]
[[(0, 0.1090186927085825), (1, 0.099510658029171292), (2, 0.10443083051301108), (3, 0.096818545489455302), (4, 0.10403733132551497), (5, 0.096666639869577853), (6, 0.097496353956395818), (7, 0.097416

[[(0, 0.1090186927085825), (1, 0.099510658029171292), (2, 0.10443083051301108), (3, 0.096818545489455302), (4, 0.10403733132551497), (5, 0.096666639869577853), (6, 0.097496353956395818), (7, 0.097416784973873599), (8, 0.089633012629827757), (9, 0.10497115050458986)], [(0, 0.10009526477830923), (1, 0.11046698331687663), (2, 0.10334391188932507), (3, 0.10965892506057254), (4, 0.098800840328416795), (5, 0.085062693158313579), (6, 0.1046500872644716), (7, 0.087054635645778963), (8, 0.096706653012677821), (9, 0.10416000554525781)], [(0, 0.093765875386151457), (1, 0.10573384818423306), (2, 0.089772386806775156), (3, 0.099044200976824023), (4, 0.097976913635631796), (5, 0.12128526535961352), (6, 0.10634355521413347), (7, 0.093109065524728737), (8, 0.10325998731066167), (9, 0.089708901601247198)], [(0, 0.091539858233709887), (1, 0.1025089089444341), (2, 0.10011127361214996), (3, 0.10386360665827828), (4, 0.095336268958828865), (5, 0.10415367313095479), (6, 0.11344865631023698), (7, 0.086497468

[[(0, 0.1090186927085825), (1, 0.099510658029171292), (2, 0.10443083051301108), (3, 0.096818545489455302), (4, 0.10403733132551497), (5, 0.096666639869577853), (6, 0.097496353956395818), (7, 0.097416784973873599), (8, 0.089633012629827757), (9, 0.10497115050458986)], [(0, 0.10009526477830923), (1, 0.11046698331687663), (2, 0.10334391188932507), (3, 0.10965892506057254), (4, 0.098800840328416795), (5, 0.085062693158313579), (6, 0.1046500872644716), (7, 0.087054635645778963), (8, 0.096706653012677821), (9, 0.10416000554525781)], [(0, 0.093765875386151457), (1, 0.10573384818423306), (2, 0.089772386806775156), (3, 0.099044200976824023), (4, 0.097976913635631796), (5, 0.12128526535961352), (6, 0.10634355521413347), (7, 0.093109065524728737), (8, 0.10325998731066167), (9, 0.089708901601247198)], [(0, 0.091539858233709887), (1, 0.1025089089444341), (2, 0.10011127361214996), (3, 0.10386360665827828), (4, 0.095336268958828865), (5, 0.10415367313095479), (6, 0.11344865631023698), (7, 0.086497468

In [92]:
# Test
x = lst[0]
print(len(x))

10


In [93]:
# Test
print(x[1])
y = x[1]
print(y[1])

(1, 0.099510658029171292)
0.0995106580292


In [94]:
# Iterating over the list to select the category that has the highest level of certainty and returning it to a list
maxes = []
for f in lst:
    mx = 0
    leng = len(f)
    counter = 0
    for x in f:
        counter += 1
        y = x[1]
        if y > mx:
            mx = y
            mxx = x
        if counter == leng:
            maxes.append(mxx)
        
print(maxes)           

[(0, 0.1090186927085825), (1, 0.11046698331687663), (5, 0.12128526535961352), (6, 0.11344865631023698), (5, 0.11379482242411358), (2, 0.11193452713759471), (9, 0.1076968639018507), (6, 0.10922626061146588), (9, 0.113047128660134), (1, 0.10783510515818968), (1, 0.10491001587955151), (5, 0.10359994473561265), (8, 0.11124619714186949), (2, 0.1056998596555731), (5, 0.10937970663193994), (2, 0.10275676934379524), (4, 0.11583753661314795), (4, 0.10818742185184416), (8, 0.10729599502787462), (9, 0.10437335867083775), (6, 0.1108840731858807), (3, 0.10415935961705715), (2, 0.11632920836459117), (1, 0.10731080201628397), (3, 0.11294343209258539), (7, 0.11019803469745075), (3, 0.10541054276291779), (1, 0.109288695802977), (9, 0.11669034647851544), (5, 0.11187524237767575), (7, 0.11051697016680381), (7, 0.11998801702238886), (1, 0.11586888842056291), (9, 0.11283550472919869), (4, 0.11886098236740698), (6, 0.11552585072669408), (3, 0.11252185300313126), (9, 0.11261884376388206), (9, 0.1268631652042

In [95]:
# Creating a list of catrgories to match with the article list
category = []
for x in maxes:
    y = x[0]
    category.append(y)
print(category)
len(category)

[0, 1, 5, 6, 5, 2, 9, 6, 9, 1, 1, 5, 8, 2, 5, 2, 4, 4, 8, 9, 6, 3, 2, 1, 3, 7, 3, 1, 9, 5, 7, 7, 1, 9, 4, 6, 3, 9, 9, 7, 3, 2, 1, 2, 6, 2, 7, 5, 1, 6, 1, 0, 4, 5, 5, 3, 3, 6, 5, 7, 9, 8, 6, 7, 1, 6, 7, 4, 7, 3, 1, 5, 8, 2, 7, 7, 5, 0, 4, 5, 2, 7, 2, 1, 8, 6, 7, 9, 8, 4, 5, 1, 8, 2, 3, 0, 3, 2, 7, 6]


100

In [96]:
# Creating dataframe 
df = pd.DataFrame()


In [97]:
# df continued
df['category'] = category
df['article'] = text_list

In [98]:
# Checking our dataframe
df.head(10)

,category,article
0,0,Article_1.txt
1,1,Article_2.txt
2,5,Article_3.txt
3,6,Article_4.txt
4,5,Article_5.txt
5,2,Article_6.txt
6,9,Article_7.txt
7,6,Article_8.txt
8,9,Article_9.txt
9,1,Article_10.txt


In [99]:
# Export to compare to article topic list in excel
df.to_csv('lda_results')

In [ ]:
# Category 4 was 100% accurate while others need more manipulation